In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpStatus, value
import pandas as pd

In [ ]:
# monkeypatching solve method to display solver output in notebook

import monkeypatch

In [ ]:
data = pd.DataFrame(
    index = ["chicken", "beef", "mutton", "rice", "wheat", "gel"],
    data = {
        "protein": [0.1, 0.2, 0.15, 0, 0.04, 0],
        "fat": [0.08, 0.1, 0.11, 0.01, 0.01, 0],
        "fibre": [0.001, 0.005, 0.003, 0.1, 0.15, 0],
        "salt": [0.002, 0.005, 0.007, 0.002, 0.008, 0],
        "cost": [0.013, 0.008, 0.01, 0.002, 0.005, 0.001],
    }
)
ingredients = data.index  # for readability
data


In [ ]:
# Creating a new minimisation problem

prob = LpProblem("The Whiskas Problem", LpMinimize)


In [ ]:
# Creating variables

ingredient_vars = LpVariable.dicts(
    name="Ingredients",
    indices = ingredients,
    lowBound=0,
)

In [ ]:
# This cell is equivalent to the one below

# objective = (
#     data.loc["chicken", "cost"] * ingredient_vars["chicken"]
#     + data.loc["beef", "cost"] * ingredient_vars["beef"]
#     + data.loc["mutton", "cost"] * ingredient_vars["mutton"]
#     + data.loc["rice", "cost"] * ingredient_vars["rice"]
#     + data.loc["wheat", "cost"] * ingredient_vars["wheat"]
#     + data.loc["gel", "cost"] * ingredient_vars["gel"]
# )

# prob += objective, "Total cost of ingredients per 100g"


In [ ]:
# Adding the objective (the total cost) to the problem

prob += (
    lpSum(data.loc[ingredient, "cost"]*ingredient_vars[ingredient] for ingredient in ingredients),
    "Total cost of ingredients per 100g"
)

In [ ]:
# Adding quantity constraint to the problem

prob += lpSum(ingredient_vars[ingredient] for ingredient in ingredients) == 100, "Total equals 100g"

In [ ]:
# Adding macronutrient constraints to the problem

prob += (
    lpSum(data.loc[ingredient, "protein"]*ingredient_vars[ingredient] for ingredient in ingredients) >= 8.0,
    "Protein Requirement",
)

prob += (
    lpSum(data.loc[ingredient, "fat"]*ingredient_vars[ingredient] for ingredient in ingredients) >= 6.0,
    "Fat Requirement",
)

prob += (
    lpSum(data.loc[ingredient, "fibre"]*ingredient_vars[ingredient] for ingredient in ingredients) <= 2.0,
    "Fibre Requirement",
)

prob += (
    lpSum(data.loc[ingredient, "salt"]*ingredient_vars[ingredient] for ingredient in ingredients) <= 0.4,
    "Salt Requirement",
)

In [ ]:
prob.solve()

In [ ]:
print("Status:", LpStatus[prob.status])

In [ ]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

In [ ]:
print("Total Cost of Ingredients per can = ", value(prob.objective))